# Time series - introduction & ARIMA models

In this example, we'll explore parts of classical time series analysis and fit a few auto regressive models.

For more details on data transformation, model selection & forecasting [visit this site](http://people.duke.edu/~rnau/whatuse.htm)

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (15, 6)

import pandas as pd
pd.options.display.max_rows = 10

In [ ]:
data = pd.read_csv('../data/AirPassengers.csv', parse_dates=True, index_col='Month',date_parser=lambda x: pd.datetime.strptime(x, '%Y-%m'))
ts = data['#Passengers']
data

We'll use a well-known air passengers time series for this example. Notice the date parsion function that allows us to build an index on this time series.

In [ ]:
plt.plot(ts)
plt.show()

Next, we will make the time series stationary. Stationarity is a useful concept for forecasting as it simplifies the theory behind and allow us to use certain models.

Time series is stationary if the following conditions are fulfilled:

1. The mean of the series should not be a function of time
2. The variance of the series should not a be a function of time
3. The covariance of the i-th term and the (i + k)-th term should not be a function of time

We can perform a statistical test to see if the time series is constant. The test is known as **Dickey Fuller** test.
We will run the test on the time series data and watch the *Test Statistic*. If it gets lower then the *Critical value*, we can reject the null hypothesis and say that the series is stationary.

In [ ]:
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose

def test_stationarity(timeseries):
        
    rolling = timeseries.rolling(center=False, window=12)        
    
    plt.plot(timeseries, color='blue',label='Original')
    plt.plot(rolling.mean(), color='red', label='Rolling Mean')
    plt.plot(rolling.std(), color='black', label = 'Rolling Std')
    plt.legend(loc='best')
    plt.title('Rolling Mean & Standard Deviation')
    plt.show()
        
    
    df_test = adfuller(timeseries, autolag='AIC')
    output = pd.Series(df_test[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
        
    for key, value in df_test[4].items():
        output['Critical Value (%s)' % key] = value
        
    print('Dickey-Fuller test:')    
    print(output)

Next, we can try *seasonal_decompose* library function that can perform trend & seasonality cocomposion at once. Let's see how stationarity test performs on **residuals** (after decomposion)

In [ ]:
def decompose(ts):
    
    decomposition = seasonal_decompose(ts)
    plt.subplot(411)
    plt.plot(ts, label='Original')
    plt.legend(loc='best')
    plt.subplot(412)
    plt.plot(decomposition.trend, label='Trend')
    plt.legend(loc='best')
    plt.subplot(413)
    plt.plot(decomposition.seasonal, label='Seasonality')
    plt.legend(loc='best')
    plt.subplot(414)
    plt.plot(decomposition.resid, label='Residuals')
    plt.legend(loc='best')
    plt.tight_layout()
    plt.show()
    
    return decomposition

In [ ]:
#TODO: test stationarity & try decompose

## Model fitting

Let's try to fit one of the models onto our data but we need to make the time series stationary before.

First, we will use popular log & difference transform to get rid of trend and time dependent variance. The stationarity test should reflect this transformation.

In [ ]:
#TODO: try to apply logarithm & shift, then test stationarity again.

..now, we will fit the ARIMA model (**A**uto**R**egressive **I**ntegrated **M**oving **A**verage) to our data. Parameters can be set by inspection of auto-correlation and partial auto-correlation plots.

After the model is fitted, we can plot it over original data. Notice that we are still in the transformed space.

In [ ]:
from statsmodels.tsa.arima_model import ARIMA

#TODO: instatiate ARIMA model and fit it. (use order=(2, 0, 2) or (2, 1, 2) based on used ts (shift or no shift)) Visualize fitted values.

model = ARIMA( ... )  
results = ...

#plt.plot(results.fittedvalues, c='r')

Last step is to inverse-transform the data back to the original domain.

As we took a log & difference, we need to apply them in reverse order (i.e. cumulative sum & exp). Also, notice that with difference we lost the constant and we need to replace it with the first value of the original dataset.

Same apply for prediction out of the sample (to the future) whre we use the last datapoint from the original time series

In [ ]:
import datetime
from datetime import timedelta

start=ts.index[-1]
end=ts.index[-1] + timedelta(days=365 * 2)

prediction = results.predict(start=ts.index[-1], end=ts.index[-1] + timedelta(days=365 * 2), dynamic=True)

#TODO: inverse-transform prediciton to the original space

There are more advanced models like SARIMAX, that separates the model for seasonal part. The usage is very similar although there are more hyper parameters to setup. One can use grid search to find to best combination of hyper parameters (not shown here).

As the model is a statistical one, The prediction plot may contain confidence intervals to see how certain the model is about its own prediction (gray area) i.e. according to the model, the prediction should be within the interval with 95% certainty.

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

#TODO: try to fit SARIMAX model